# Import

In [1]:
# System
import os
import time

os.environ["OMP_NUM_THREADS"] = "4" # export OMP_NUM_THREADS=4
os.environ["OPENBLAS_NUM_THREADS"] = "4" # export OPENBLAS_NUM_THREADS=4 
os.environ["MKL_NUM_THREADS"] = "6" # export MKL_NUM_THREADS=6
os.environ["VECLIB_MAXIMUM_THREADS"] = "4" # export VECLIB_MAXIMUM_THREADS=4
os.environ["NUMEXPR_NUM_THREADS"] = "6" # export NUMEXPR_NUM_THREADS=6

# Data processing
import numpy as np
import netCDF4 as nc
import math


# Plot
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter
from cartopy.mpl.ticker import LatitudeFormatter

# ML
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# Config

## Config (environment)

In [2]:
config = {}

# Fname 
config.update({"FolderName": "2023_0310"})

# Fpath
config.update({"FolderPath": "/Volumes/Expansion/User_Backup/b08209033/111-2_IVT_analysis"})
config.update({"SubFolderPath": os.path.join(config["FolderPath"], config["FolderName"])})
config.update({"DataPath": os.path.join(config["FolderPath"], "data")})
config.update({"SrcPath": os.path.join(config["SubFolderPath"], "src")})
config.update({"ImgPath": os.path.join(config["SubFolderPath"], "img")})

os.chdir(config["FolderPath"])

## Config (ML)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
configML = {
    'seed': 9527,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 100,     # Number of epochs.            
    'batch_size': 1, 
    'learning_rate': 1e-5,
    'weight_decay': 0,
    'early_stop': 10,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}
print(device)

cpu


# Initialize environment

In [4]:
def createFolder(path):
    if not os.path.exists(path):
        os.makedirs(path)
    #else:
    #    print("Folder already existed.")
createFolder(config["FolderPath"])
createFolder(config["SrcPath"])
createFolder(config["ImgPath"])
createFolder(config["DataPath"])

# Self-defiend Objects

## IVT

In [5]:
# Input .nc object
# Return IVT with dimensions (year, day, level, lat, lon)
def getIVT(rootgrp, remove_trend=False):
    
    # Determine array shape
        # Find region index
    lon = rootgrp['lon'][:]
    lat = rootgrp['lat'][:]
        # Find level index
    level = rootgrp['plev'][:]
    
    # Read variables in original file
    u = rootgrp['u'][:,:,:,:].reshape(-1, 365, level.shape[0], lat.shape[0], lon.shape[0])
    v = rootgrp['v'][:,:,:,:].reshape(-1, 365, level.shape[0], lat.shape[0], lon.shape[0])
    q = rootgrp['q'][:,:,:,:].reshape(-1, 365, level.shape[0], lat.shape[0], lon.shape[0])
    if (remove_trend != False):
        # Read variables in long-time average trend file
        u_trend = remove_trend['u'][:,:,:,:]
        v_trend = remove_trend['v'][:,:,:,:]
        q_trend = remove_trend['q'][:,:,:,:]
    
    # Calculate IVT
        # Calculate interpolated value by averaging
    interp_u = (u[:,:,1:,:,:] + u[:,:,:-1,:,:])/2
    interp_v = (v[:,:,1:,:,:] + v[:,:,:-1,:,:])/2
    interp_q = (q[:,:,1:,:,:] + q[:,:,:-1,:,:])/2
        # hPa to Pa unit conversion
    thickness = (level[:-1] - level[1:])
        # Broadcast 
    thickness = thickness[np.newaxis, np.newaxis, :, np.newaxis, np.newaxis]
        # Integrate over levels
    IVT_u = np.sum((interp_u * interp_q * thickness)/9.81, axis = 2)
    IVT_v = np.sum((interp_v * interp_q * thickness)/9.81, axis = 2)
    
    return lat, lon, np.array((IVT_u, IVT_v))

## Dataset

In [6]:
class IVTDataset(torch.utils.data.Dataset):
    # x, y is defined over lead-lag-1
    def __init__(self, x, y=None, seq_len=14):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)
        self.seq_len = seq_len
    """
    Modify here to implement seq_len
    """
    # Using sequence to get window
    # Padding if exceeded
    def __getitem__(self, idx):
        if idx + self.seq_len > self.__len__():
            x_pad = torch.zeros(self.seq_len, self.x.size()[1])
            y_pad = torch.zeros(self.seq_len, self.y.size()[1])
            x_pad[:self.__len__()-idx,:] = self.x[idx:]
            y_pad[:self.__len__()-idx,:] = self.y[idx:]
            return x_pad, y_pad
        else:
            return self.x[idx:idx+self.seq_len], self.y[idx:idx+self.seq_len]
    def __len__(self):
        return len(self.x)

# Model

In [7]:
class My_Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(My_Model, self).__init__()
        self.GRU = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.input_length = input_size
        self.num_layers = num_layers
        self.hidden_dim = hidden_size
        
    def forward(self, x, hidden_state):
        output, hidden_state = self.GRU(x, hidden_state)
        return output, hidden_state
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.num_layers, batch_size, self.hidden_dim).zero_()
        return hidden

# Data preprocessing

In [8]:
# Defined dims
TOTAL_DIM = 2
TOTAL_YEAR = 43

# Process
count = time.time()
os.chdir(config["DataPath"])
rootgrp = nc.Dataset("dataset.nc")
trend = nc.Dataset("seasonal_cycle.nc")
lat, lon, IVT = getIVT(rootgrp, trend)
#lat, lon, IVT = getIVT(rootgrp)
rootgrp.close()
trend.close()
print(f"Takes {(time.time()-count):.3f} sec")

Takes 16.644 sec


# Data processing (SVD)

In [9]:
count = time.time()
# Time axis reshape
data = IVT.reshape(TOTAL_DIM, -1, lat.shape[0], lon.shape[0])
# Space axis reshape
data = data.reshape(TOTAL_DIM, TOTAL_YEAR*365, lat.shape[0]*lon.shape[0])
# Variable axis reshape
data = np.concatenate((data[0,:,:],data[1,:,:]), axis = -1)
# Transpose to (Space, Time) dimension
data = data.transpose()
# SVD
u, s, vh = np.linalg.svd(data, full_matrices=False)
print(f"Takes {(time.time()-count):.3f} sec")

Takes 222.294 sec


# Variance (Explainability)

In [10]:
# Defined threshold
threshold = 0.9

# Calculate explainability
variance = np.square(s)
total_var = np.sum(variance)
ith_var = 0
feature_num = None
for i, var_i in enumerate(variance):
    ith_var += var_i/total_var
    if (ith_var >= threshold):
        print(f"First {i+1} components explain {ith_var*100:.2f}% variance.")
        feature_num = i+1
        break
space_weights = np.copy((vh[:feature_num]))
scalar_parameter_a = np.min(space_weights)
scalar_parameter_b = np.max(space_weights) - np.min(space_weights)
space_weights = (space_weights - scalar_parameter_a) / scalar_parameter_b

First 17 components explain 90.17% variance.


# Undetermined blocks

In [11]:
# Defined train valid test split
train_ratio = 32
valid_ratio = 3
test_ratio = 8
seq_len = 7
# Split train_valid_test
train_space_weights = (space_weights.T)[:365*train_ratio]
valid_space_weights = (space_weights.T)[365*train_ratio:365*(train_ratio+valid_ratio)]
test_space_weights = (space_weights.T)[365*(train_ratio+valid_ratio):]
# Split x-y
X_train, Y_train = train_space_weights[:-1], train_space_weights[1:]
X_valid, Y_valid = valid_space_weights[:-1], valid_space_weights[1:]
X_test, Y_test = test_space_weights[:-1], test_space_weights[1:]

# Encapsulate to class
train_dataset = IVTDataset(X_train, Y_train, seq_len = seq_len)
valid_dataset = IVTDataset(X_valid, Y_valid, seq_len = seq_len)
test_dataset = IVTDataset(X_test, Y_test, seq_len = seq_len)
# Encapsulate to Dataloader
train_loader = DataLoader(train_dataset, batch_size = configML['batch_size'], pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size = configML['batch_size'], pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size = configML['batch_size'], pin_memory=True)

In [12]:
def trainer(train_loader, valid_loader, model, config, device):
    
    # Pre train stage
        # model_dict
    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.
        # train parameter
    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0
    
    # Loss function
    criterion = nn.L1Loss(reduction='mean').to(device)
    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr = config['learning_rate'], weight_decay = config['weight_decay'])
    # Thread numbers
     
    
    # Training & Validating
    mean_train_loss_record = np.array([])
    mean_valid_loss_record = np.array([])
    for epoch in range(n_epochs):
        # Training stage
        # Init
        model.train()
        loss_record = []
        hidden = model.init_hidden(config['batch_size'])
        for x, y in train_loader:
            # Reset gradient
            optimizer.zero_grad()
            # cuda if possible
            x, y = x.to(device), y.to(device)
            # Modify batch size in hidden state
            if(x.size()[0]!=hidden.size()[1]):
                hidden = (hidden[:,hidden.size()[1]-x.size()[0]:,:])
            # Forward propagation
            pred, hidden = model(x, hidden)
            # Calculate loss
            loss = criterion(pred, y)
            # Backward propagation
            loss.backward()
            # Update model parameter
            optimizer.step()                    
            step += 1
            # Detach unused graph
            hidden.detach_()
            loss_record.append(loss.detach().item())
        
        # Train loss
        mean_train_loss = sum(loss_record)/len(loss_record)
        
        # Validating stage
        # Init
        model.eval() # Set your model to evaluation mode.
        loss_record = []
        hidden = model.init_hidden(configML['batch_size'])
        for x, y in valid_loader:
            # cuda if possible
            x, y = x.to(device), y.to(device)
            # Modify batch size in hidden state
            if(x.size()[0]!=hidden.size()[1]):
                hidden = (hidden[:,hidden.size()[1]-x.size()[0]:,:])
            # Skip gradient update and backward propagation
            with torch.no_grad():
                # Forward propagation
                pred, hidden = model(x, hidden)
                # Calculate loss
                loss = criterion(pred, y)
            
            # Detach loss
            loss_record.append(loss.item())
            
        # Valid loss
        mean_valid_loss = sum(loss_record)/len(loss_record)
        
        # Show progress
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.7f}, Valid loss: {mean_valid_loss:.7f}')
        
        # Save model parameter
        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.5f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1
            
        # Early stop
        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return


In [13]:
os.chdir(config["FolderPath"])
model = My_Model(input_size = feature_num,
                 hidden_size = feature_num,
                 num_layers = 1).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, configML, device)

Epoch [1/100]: Train loss: 0.1797925, Valid loss: 0.0606732
Saving model with loss 0.06067...
Epoch [2/100]: Train loss: 0.0586119, Valid loss: 0.0599759
Saving model with loss 0.05998...
Epoch [3/100]: Train loss: 0.0579648, Valid loss: 0.0593973
Saving model with loss 0.05940...
Epoch [4/100]: Train loss: 0.0573958, Valid loss: 0.0588710
Saving model with loss 0.05887...
Epoch [5/100]: Train loss: 0.0568656, Valid loss: 0.0583732
Saving model with loss 0.05837...
Epoch [6/100]: Train loss: 0.0563659, Valid loss: 0.0579118
Saving model with loss 0.05791...
Epoch [7/100]: Train loss: 0.0558904, Valid loss: 0.0574849
Saving model with loss 0.05748...
Epoch [8/100]: Train loss: 0.0554318, Valid loss: 0.0570782
Saving model with loss 0.05708...
Epoch [9/100]: Train loss: 0.0549827, Valid loss: 0.0566736
Saving model with loss 0.05667...
Epoch [10/100]: Train loss: 0.0545392, Valid loss: 0.0562749
Saving model with loss 0.05627...
Epoch [11/100]: Train loss: 0.0541016, Valid loss: 0.055876

In [16]:
model.eval() # Set your model to evaluation mode.
preds = []
hidden = model.init_hidden(configML['batch_size'])
for i, (x,y) in enumerate(test_loader):
    x = x.to(device)                        
    with torch.no_grad():
        pred, hidden = model(x, hidden)
        preds.append(pred.detach().cpu()[:,0,:])
preds = torch.cat(preds, dim=0).numpy()
print(x)

tensor([[[0.4421, 0.4003, 0.4769, 0.3974, 0.4366, 0.5368, 0.4437, 0.5206,
          0.4685, 0.5146, 0.4882, 0.4545, 0.4595, 0.4734, 0.3668, 0.5476,
          0.5435],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000]

In [15]:
os.chdir(config["ImgPath"])

for idx in range(feature_num):
    plt.figure(figsize=(12,8), dpi = 200)
    plt.plot(preds[:,idx], label = "predict", zorder = 3)
    plt.plot(Y_test[:,idx], label = "target", zorder = 2)
    plt.xticks(np.arange(180,2920,365))
    plt.legend(loc = 1, prop={'size': 20})
    plt.title(f"Time weight of filtered SVD Spatial mode:{idx}")
    plt.savefig(f"Time series, mode:{idx}")
    plt.close()

# Unused blocks